# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010 driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 Championship was won by Fernando Alonso, driving for the Renault team.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Sorry, I cannot provide an answer to that question as it is outside the scope of the information provided in this conversation.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## MathBot - Solving Math Equations 🧮
- Objective: Test if the model correctly solves equations and explains steps clearly.

In [14]:
messages = [
    {'role': 'system', 'content': 'You are a math expert. Solve mathematical equations accurately with a step-by-step explanation.'},
    {'role': 'user', 'content': 'Solve the equation: 2x + 5 = 15'}
]
response = return_OAIResponse("Solve the equation: 2x + 5 = 15", messages)
print(response)

To solve the equation 2x + 5 = 15, we need to isolate the variable x.

First, we'll subtract 5 from both sides of the equation:  
2x + 5 - 5 = 15 - 5  
2x = 10

Next, we'll divide both sides of the equation by 2 to solve for x:  
2x/2 = 10/2  
x = 5

Therefore, the solution to the equation 2x + 5 = 15 is x = 5.


## DoctorBot - Providing Medical Advice 🏥
- Objective: Test the accuracy and safety of AI-generated medical advice.

In [15]:
messages = [
    {'role': 'system', 'content': 'You are a doctor. Provide health advice based on scientific evidence without giving direct diagnoses.'},
    {'role': 'user', 'content': 'I have a persistent headache, what should I do?'}
]
response = return_OAIResponse("I have a persistent headache, what should I do?", messages)
print(response)

If you are experiencing persistent headaches, it is important to consult with a healthcare professional for proper evaluation and diagnosis. Persistent headaches can be caused by various factors such as stress, dehydration, lack of sleep, eye strain, or underlying health conditions. Keeping a headache diary to track your symptoms and potential triggers may also be helpful. In the meantime, you can try to manage your headaches by practicing good sleep hygiene, staying hydrated, managing stress through relaxation techniques, and maintaining a healthy diet. Over-the-counter pain relievers may provide temporary relief, but it is important to address the underlying cause with guidance from a healthcare provider.


## TravelBot - Suggesting Tourist Destinations ✈️
- Objective: Evaluate the relevance and accuracy of travel recommendations.

In [16]:
messages = [
    {'role': 'system', 'content': 'You are a travel expert. Recommend travel destinations based on the user’s interests and season.'},
    {'role': 'user', 'content': 'I want to visit a European country in winter. Any suggestions?'}
]
response = return_OAIResponse("I want to visit a European country in winter. Any suggestions?", messages)
print(response)

If you're looking to visit a European country in winter, here are some destinations you might consider:

1. **Austria**: Experience the charm of cities like Vienna and Salzburg, go skiing in the Alps, and attend traditional Christmas markets.

2. **Switzerland**: Enjoy the breathtaking landscapes of the Swiss Alps, explore picturesque villages like Zermatt and Interlaken, and indulge in Swiss chocolate.

3. **Norway**: Witness the Northern Lights in Tromso, go dog sledding in Alta, and explore the stunning fjords along the western coast.

4. **Finland**: Visit the magical Lapland region, meet Santa Claus in Rovaniemi, go ice fishing on frozen lakes, and relax in traditional Finnish saunas.

5. **Czech Republic**: Explore the fairytale city of Prague, enjoy winter sports in the Bohemian Forest, and stroll through festive Christmas markets.

6. **Germany**: Experience the festive atmosphere at Christmas markets in cities like Berlin, Munich, and Nuremberg, and enjoy winter sports in the 

## Key Takeaways from These New Topics
- Assigning a specific role (Math Expert, Doctor, Travel Guide) improves response accuracy.
- Setting clear boundaries (e.g., "Do not provide a diagnosis") enhances safety.
- Testing edge cases helps identify and correct AI-generated mistakes.

## Standard Question with Context

In [11]:
messages = [
    {'role': 'system', 'content': 'You are an AI assistant helping with science questions.'},
    {'role': 'user', 'content': 'What is the speed of light in vacuum?'}
]
response = return_OAIResponse("What is the speed of light in vacuum?", messages)
print(response)


The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s) or about 186,282 miles per second.


## Role-Based Prompting

In [12]:
messages = [
    {'role': 'system', 'content': 'You are a physics professor. Explain concepts simply and clearly.'},
    {'role': 'user', 'content': 'Explain quantum entanglement in simple terms.'}
]
response = return_OAIResponse("Explain quantum entanglement in simple terms.", messages)
print(response)


Quantum entanglement is a strange and fascinating behavior that happens between particles, such as atoms or photons. When two particles are entangled, their properties become linked in such a way that the state of one particle can instantly affect the state of the other particle, no matter how far apart they are.

Imagine you have two entangled particles: Particle A and Particle B. If you measure a property of Particle A, like its spin or polarization, the act of measuring instantly determines the corresponding property of Particle B, even if it's far away. This spooky action at a distance happens faster than the speed of light and seems to violate our classical understanding of physics.

Scientists are still studying quantum entanglement to unlock its full potential for technologies like quantum computing and quantum communication.


## Testing Edge Cases (Ambiguous Request)

In [13]:
messages = [
    {'role': 'system', 'content': 'You are a historian specializing in ancient civilizations.'},
    {'role': 'user', 'content': 'Tell me about the pyramids.'}
]
response = return_OAIResponse("Tell me about the pyramids.", messages)
print(response)


The pyramids are large, monumental structures built by the ancient Egyptians as tombs for their pharaohs and queens. The most famous pyramids are located in Giza, near Cairo, and were constructed during the Old Kingdom period (circa 2686–2181 BCE).

The Great Pyramid of Giza, built for Pharaoh Khufu, is the largest and most iconic pyramid. It was originally 146.6 meters tall, making it the tallest man-made structure in the world for over 3,800 years. The precise methods used to construct the pyramids remain a topic of debate among historians and archaeologists.

Pyramids were built using massive limestone blocks, quarried locally and transported to the construction site. The blocks were precisely cut and fitted together to form the pyramid's shape. Inside the pyramids, there were various chambers and tunnels, including the burial chamber where the pharaoh's sarcophagus was placed.

The pyramids were designed as part of a larger funerary complex, which included mortuary temples, causewa

## Findings & Observations
- Structured prompts improve accuracy.
- Ambiguous questions lead to biased or incomplete responses.
- Without context, the model may misunderstand the user’s intent.

## Lessons Learned
- Providing a role-based instruction improves response clarity.
- Setting a clear scope (e.g., “Explain for beginners”) helps target the right audience.
- Lowering randomness (temperature=0.2) reduces hallucinations and increases reliability.